In [1]:
import nltk
import random
import feedparser

In [2]:
urls = {
    'mlb': 'https://sports.yahoo.com/mlb/rss.xml',
    'nfl': 'https://sports.yahoo.com/nfl/rss.xml',
}

In [17]:
feedmap = {}

In [18]:
stopwords = nltk.corpus.stopwords.words('english') # stopword : 불용어

In [19]:
def featureExtractor(words):
    features = {}
    for word in words:
        if word not in stopwords: # 단어 목록을 가져와 딕셔너리에 추가
            features["word({})".format(word)] = True
    return features

In [20]:
sentences = []

In [21]:
for category in urls.keys(): # 중첩 반복문  
    feedmap[category] = feedparser.parse(urls[category])
    print("downloading {}".format(urls[category]))
    
    for entry in feedmap[category]['entries']:
        data = entry['summary']
        words = data.split()
        sentences.append((category, words))

downloading https://sports.yahoo.com/mlb/rss.xml
downloading https://sports.yahoo.com/nfl/rss.xml


In [22]:
featuresets = [(featureExtractor(words), category) for category, words in sentences]

In [23]:
random.shuffle(featuresets)

In [24]:
total = len(featuresets)
off = int(total/2)
trainset = featuresets[off:]
testset = featuresets[:off]

In [25]:
classifier = nltk.NaiveBayesClassifier.train(trainset)

In [27]:
print(nltk.classify.accuracy(classifier, testset)) # 정확도 출력

0.8235294117647058


In [28]:
classifier.show_most_informative_features(5)

Most Informative Features
            word(Sunday) = True              nfl : mlb    =      4.9 : 1.0
              word(game) = True              nfl : mlb    =      4.9 : 1.0
               word(The) = True              nfl : mlb    =      3.7 : 1.0
            word(Tigers) = True              mlb : nfl    =      3.3 : 1.0
            word(season) = True              nfl : mlb    =      2.6 : 1.0


In [29]:
for (i, entry) in enumerate(feedmap['nfl']['entries']):
    if i < 4:
        features = featureExtractor(entry['title'].split()) # title을 기준으로 태깅
        category = classifier.classify(features)
        print('{} -> {}'.format(category, entry['summary']))

nfl -> With the era of autocrat coaches cycling out and a new age of younger minds coming in, college coaches could be more appealing to NFL franchises.
nfl -> Here are four keys to follow to give them better odds.
nfl -> Even though this game won't be the most exciting, experts are picking the Detroit Lions to beat the Washington Football Team this week.
nfl -> The Ravens and Patriots have developed a potent rivalry over the past decade, with multiple playoff meetings and still-lingering suspicion in New England that Baltimore instigated #Deflategate. This year, the game lacks some sizzle, because the Patriots for the first time in a long time lack the skill to contend with the best teams in [more]
